In [ ]:
pip install matplotlib

In [ ]:
import time
import random
import numpy as np
import threading
import matplotlib
import matplotlib.pyplot as plt
import tkinter as tk
from PIL import ImageTk, Image
from collections import defaultdict

In [ ]:
np.random.seed(1)
PhotoImage = ImageTk.PhotoImage
UNIT = 50  # number of pixel
HEIGHT = 16  # height of grid world
WIDTH = 16  # width of grid world

In [ ]:
class Env(tk.Tk):
    def __init__(self):
        super(Env, self).__init__()
        # set action space
        self.action_space = ['up', 'down', 'left', 'right',
                             'bus103', 'bus104', 'bus151', 'bus173', 'bus201', 'bus301', 'bus704', 'bus1102', 'bus7022', 
                             'metro 1', 'metro 2', 'metro 3', 'metro 4', 'metro 5'] 
        self.n_actions = len(self.action_space)
        self.title('Sarsa')
        self.geometry('{0}x{1}'.format(HEIGHT * UNIT, HEIGHT * UNIT))
        self.shapes = self.load_images()
        self.texts = []
        # station of each bus and metro
        self.bus103 = [[175, 775], [125, 575], [125, 425], [275, 425], [325, 425], [425, 425], [475, 425], [575, 425], [625, 425], [725, 425]]
        self.bus104 = [[175, 775], [275, 675], [425, 625], [525, 625], [675, 575], [675, 525], [675, 475], [675, 425], [675, 375], [675, 325], 
                       [675, 275], [375, 175]]
        self.bus151 = [[175, 775], [275, 625], [325, 575], [375, 475], [375, 375], [375, 275], [475, 275], [575, 225], [625, 125]]
        self.bus173 = [[175, 775], [225, 675], [275, 625], [325, 575], [375, 475], [475, 475], [525, 475], [625, 475], [675, 425], [725, 425], [775, 425]]
        self.bus201 = [[175, 775], [275, 625], [325, 575], [375, 475], [425, 425], [475, 425], [575, 425], [625, 425], [725, 425]]
        self.bus301 = [[775, 675], [725, 625], [775, 575], [775, 525], [775, 325], [725, 275], [725, 275], [675, 175], [675, 75], [625, 125], 
                       [575, 225], [675, 275], [775, 375], [775, 525], [775, 575], [725, 625], [775, 675]]
        self.bus704 = [[175, 775], [175, 575], [75, 475]]
        self.bus1102 = [[175, 775], [425, 425], [675, 175]]
        self.bus7022 = [[175, 75], [175, 125], [125, 175], [175, 225], [175, 275], [325, 275], [275, 325], [225, 475], [225, 575], [175, 775], 
                        [225, 675], [275, 625], [325, 575], [375, 475], [375, 375]]
        self.metro1 = [[125, 775], [275, 525], [375, 425], [525, 375], [675, 375], [775, 375]]
        self.metro2 = [[25, 625], [375, 525], [325, 475], [525, 475], [625, 475], [775, 525]]
        self.metro3 = [[175, 275], [375, 275], [525, 375], [525, 475], [575, 625], [725, 675]]
        self.metro4 = [[125, 775], [275, 675], [425, 625], [575, 625], [775, 525], [775, 375], [675, 175], [725, 25]]
        self.metro5 = [[75, 475], [225, 375], [525, 375], [625, 475], [775, 575]]
        self.bus_routes = [self.bus103, self.bus104, self.bus151, self.bus173, self.bus201, self.bus301, self.bus704, self.bus1102, self.bus7022]
        self.metro_routes = [self.metro1, self.metro2, self.metro3, self.metro4, self.metro5]
        self.stateList = self.bus_routes + self.metro_routes
        self.canvas = self._build_canvas()
        
    def _build_canvas(self):
        canvas = tk.Canvas(self, bg='white',
                           height=HEIGHT * UNIT,
                           width=WIDTH * UNIT)
        
        # make grid world
        for c in range(0, WIDTH * UNIT, UNIT): 
            x0, y0, x1, y1 = c, 0, c, HEIGHT * UNIT
            canvas.create_line(x0, y0, x1, y1)
        for r in range(0, HEIGHT * UNIT, UNIT):  
            x0, y0, x1, y1 = 0, r, HEIGHT * UNIT, r
            canvas.create_line(x0, y0, x1, y1)

        # add images
        self.map = canvas.create_image(400, 400, image=self.shapes[5])
        self.people = canvas.create_image(125, 775, image=self.shapes[0])
        self.x1 = canvas.create_image(675, 175, image=self.shapes[1])
        self.x2 = canvas.create_image(675, 225, image=self.shapes[1])
        self.x3 = canvas.create_image(225, 275, image=self.shapes[1])
        self.x4 = canvas.create_image(525, 625, image=self.shapes[1])
        self.x5 = canvas.create_image(575, 625, image=self.shapes[1])
        self.x6 = canvas.create_image(175, 725, image=self.shapes[1])
        self.school = canvas.create_image(575, 75, image=self.shapes[2])
        self.bus = canvas.create_image(125, 775, image=self.shapes[3])
        self.metro = canvas.create_image(125, 775, image=self.shapes[4])
        

        canvas.pack()

        return canvas

    def load_images(self):
        people = PhotoImage(
            Image.open("people.jpg").resize((50, 50)))
        triangle = PhotoImage(
            Image.open("x.png").resize((35, 35)))
        school = PhotoImage(
            Image.open("skku.png").resize((35, 35)))
        bus = PhotoImage(
            Image.open("bus.png").resize((35, 35)))
        metro = PhotoImage(
            Image.open("metro.png").resize((35, 35)))
        map = PhotoImage(
            Image.open("map.png").resize((800, 800)))

        return people, triangle, school, bus, metro, map

    def text_value(self, row, col, contents, font='Helvetica', size=10,
                   style='normal', anchor="nw"):
        origin_x, origin_y = 7, 7
        x, y = origin_y + (UNIT * col), origin_x + (UNIT * row)
        font = (font, str(size), style)
        text = self.canvas.create_text(x, y, fill="black", text=contents,
                                        font=font, anchor=anchor)

        return self.texts.append(text)

    def print_value_all(self, q_table):
        for i in self.texts:
            self.canvas.delete(i)
        self.texts.clear()
        for i in range(HEIGHT):
            for j in range(WIDTH):
                for action in range(0, 18):
                    state = [i, j]
                    if str(state) in q_table.keys():
                        temp = q_table[str(state)][action]
                        self.text_value(j, i, round(temp, 2), action)


    def coords_to_state(self, coords):
        x = int((coords[0] - 25) / 50)
        y = int((coords[1] - 25) / 50)
        return [x, y]

    def state_to_coords(self, state):
        x = int(state[0] * 50 + 25)
        y = int(state[1] * 50 + 25)
        return [x, y]

    def reset(self):
        self.update()
        time.sleep(0.5)
        x, y = self.canvas.coords(self.people)
        
        self.canvas.move(self.people, -x + 125, -y + 775)
        self.canvas.move(self.bus, -x + 125, -y + 775)
        self.canvas.move(self.metro, -x + 125, -y + 775)
        self.render()
        return self.coords_to_state(self.canvas.coords(self.people))

    def step(self, action):
        state = self.canvas.coords(self.people)
        base_action = np.array([0, 0])
        self.render()

        if action == 0:  
            if state[1] > UNIT:
                base_action[1] -= UNIT
        elif action == 1:
            if state[1] < (HEIGHT - 1) * UNIT:
                base_action[1] += UNIT
        elif action == 2:  
            if state[0] > UNIT:
                base_action[0] -= UNIT
        elif action == 3:  
            if state[0] < (WIDTH - 1) * UNIT:
                base_action[0] += UNIT
        elif action == 4:
            state_index = self.bus103.index(state)
            if state_index != len(self.bus103) - 1:
                base_action = self.bus103[state_index + 1]
            else: 
                base_action = self.bus103[state_index]
        elif action == 5:
            state_index = self.bus104.index(state)
            if state_index != len(self.bus104) - 1:
                base_action = self.bus104[state_index + 1]
            else:
                base_action = self.bus104[state_index]
        elif action == 6:
            state_index = self.bus151.index(state)
            if state_index != len(self.bus151) - 1:
                base_action = self.bus151[state_index + 1]
            else:
                base_action = self.bus151[state_index]
        elif action == 7:
            state_index = self.bus173.index(state)
            if state_index != len(self.bus173) - 1:
                base_action = self.bus173[state_index + 1]
            else:
                base_action = self.bus173[state_index]
        elif action == 8:
            state_index = self.bus201.index(state)
            if state_index != len(self.bus201) - 1:
                base_action = self.bus201[state_index + 1]
            else:
                base_action = self.bus201[state_index]
        elif action == 9:
            state_index = self.bus301.index(state)
            if state_index != len(self.bus301) - 1:
                base_action = self.bus301[state_index + 1]
            else:
                base_action = self.bus301[state_index]
        elif action == 10:
            state_index = self.bus704.index(state)
            if state_index != len(self.bus704) - 1:
                base_action = self.bus704[state_index + 1]
            else:
                base_action = self.bus704[state_index]
        elif action == 11:
            state_index = self.bus1102.index(state)
            if state_index != len(self.bus1102) - 1:
                base_action = self.bus1102[state_index + 1]
            else:
                base_action = self.bus1102[state_index]
        elif action == 12:
            state_index = self.bus7022.index(state)
            if state_index != len(self.bus7022) - 1:
                base_action = self.bus7022[state_index + 1]
            else:
                base_action = self.bus7022[state_index]
        elif action == 13:
            state_index = self.metro1.index(state)
            if state_index != len(self.metro1) - 1:
                base_action = self.metro1[state_index + 1]
            else:
                base_action = self.metro1[state_index]
        elif action == 14:
            state_index = self.metro2.index(state)
            if state_index != len(self.metro2) - 1:
                base_action = self.metro2[state_index + 1]
            else:
                base_action = self.metro2[state_index]
        elif action == 15:
            state_index = self.metro3.index(state)
            if state_index != len(self.metro3) - 1:
                base_action = self.metro3[state_index + 1]
            else:
                base_action = self.metro3[state_index]
        elif action == 16:
            state_index = self.metro4.index(state)
            if state_index != len(self.metro4) - 1:
                base_action = self.metro4[state_index + 1]
            else:
                base_action = self.metro4[state_index]
        elif action == 17:
            state_index = self.metro5.index(state)
            if state_index != len(self.metro5) - 1:
                base_action = self.metro5[state_index + 1]
            else:
                base_action = self.metro5[state_index]
        
                
        if action < 4: #walk
            self.canvas.move(self.people, base_action[0], base_action[1])
            self.canvas.move(self.bus, base_action[0], base_action[1])
            self.canvas.move(self.metro, base_action[0], base_action[1])
            self.canvas.tag_raise(self.people)
            next_state = self.canvas.coords(self.people)
        elif action >= 4 and action < 13: #bus
            self.canvas.move(self.people, base_action[0]-state[0], base_action[1]-state[1])
            self.canvas.move(self.bus, base_action[0]-state[0], base_action[1]-state[1])
            self.canvas.move(self.metro, base_action[0]-state[0], base_action[1]-state[1])
            self.canvas.tag_raise(self.bus)
            next_state = self.canvas.coords(self.people)
        else: #metro
            self.canvas.move(self.people, base_action[0]-state[0], base_action[1]-state[1])
            self.canvas.move(self.bus, base_action[0]-state[0], base_action[1]-state[1])
            self.canvas.move(self.metro, base_action[0]-state[0], base_action[1]-state[1])
            self.canvas.tag_raise(self.metro)
            next_state = self.canvas.coords(self.people)
            
        
        if action < 4: #walk reward
            if next_state == self.canvas.coords(self.school): #goal
                reward_time = -500
                reward_cost = 0
                done = True
            elif next_state in [self.canvas.coords(self.x1),
                                self.canvas.coords(self.x2),
                                self.canvas.coords(self.x3),
                                self.canvas.coords(self.x4),
                                self.canvas.coords(self.x5),
                                self.canvas.coords(self.x6)]: #obstacle
                reward_time = -500
                reward_cost = 0
                done = False
            else: #normal
                reward_time = -500
                reward_cost = 0
                done = False
        elif action >= 4 and action < 13: # bus reward
            if next_state == self.canvas.coords(self.school): #goal
                reward_time = -2
                reward_cost = -1500
                done = True
            elif next_state in [self.canvas.coords(self.x1),
                                self.canvas.coords(self.x2),
                                self.canvas.coords(self.x3),
                                self.canvas.coords(self.x4),
                                self.canvas.coords(self.x5),
                                self.canvas.coords(self.x6)]: #obstacle
                reward_time = -500
                reward_cost = -1500
                done = False
            else: #normal
                reward_time = -2
                reward_cost = -1500
                done = False
        elif action >= 13: # metro reward
            if next_state == self.canvas.coords(self.school): #goal
                reward_time = -3
                reward_cost = -1400
                done = True
            elif next_state in [self.canvas.coords(self.x1),
                                self.canvas.coords(self.x2),
                                self.canvas.coords(self.x3),
                                self.canvas.coords(self.x4),
                                self.canvas.coords(self.x5),
                                self.canvas.coords(self.x6)]: #obstacle
                reward_time = -500
                reward_cost = -1400
                done = False
            else: #normal
                reward_time = -3
                reward_cost = -1400
                done = False
            
        reward = (reward_time / 2) + (reward_cost / 2)
        next_state = self.coords_to_state(next_state)
        self.render()
        return next_state, reward, reward_time, reward_cost, done

        
    def render(self):
        time.sleep(0.03)
        self.update()

In [ ]:
class SARSAgent:
    def __init__(self, actions):
        self.actions = actions
        self.learning_rate = 0.01
        self.discount_factor = 0.9
        self.epsilon = 0.1
        self.q_table = defaultdict(lambda: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
                                            0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
                                            0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
        self.actionlist = [0, 1, 2, 3]
        self.nextactionlist = [0, 1, 2, 3]
        
    # update Q function with sample <s, a, r, s', a'>
    def learn(self, state, action, reward, next_state, next_action):
        current_q = self.q_table[state][action]
        next_state_q = self.q_table[next_state][next_action]
        new_q = (current_q + self.learning_rate *
                (reward + self.discount_factor * next_state_q - current_q))
        self.q_table[state][action] = new_q

    # select action
    def get_action(self, state, actionlist):
        action = 100
        while action not in actionlist:
            if np.random.rand() < self.epsilon:
                action = np.random.choice(self.actionlist)
            else:
                state_action = self.q_table[state]
                action = self.arg_max(state_action)
        return action

    @staticmethod
    def arg_max(state_action):
        max_index_list = []
        max_value = state_action[0]
        for index, value in enumerate(state_action):
            if value > max_value:
                max_index_list.clear()
                max_value = value
                max_index_list.append(index)
            elif value == max_value:
                max_index_list.append(index)
        return random.choice(max_index_list)


In [ ]:
if __name__ == "__main__":
    env = Env()
    agent = SARSAgent(actions=list(range(env.n_actions)))
    episodelist = []
    timerewardList = []
    costrewardList = []
    rewardList = []

    for episode in range(30): # set number of episodes
        state = env.reset()
        for i in range(len(env.stateList)):
            if env.state_to_coords(state) in env.stateList[i]:
                agent.actionlist.append(i + 4)
            
        print(agent.actionlist)
        action = agent.get_action(str(state), agent.actionlist)

        done = False
        print("-----Episode ", episode+1, "-----")
        while not done:
            env.render()

            next_state, reward, reward_time, reward_cost, done = env.step(action)
            for i in range(len(env.stateList)):
                if env.state_to_coords(next_state) in env.stateList[i]:
                    agent.nextactionlist.append(i + 4)
                
            print(agent.nextactionlist)
            next_action = agent.get_action(str(next_state), agent.nextactionlist)

            agent.learn(str(state), action, reward, str(next_state), next_action)

            state = next_state
            action = next_action

            env.print_value_all(agent.q_table)
            agent.actionlist = [0, 1, 2, 3]
            agent.nextactionlist = [0, 1, 2, 3]
            print("episode: ", episode, "state: ", state, "  reward time: ", reward_time, "  reward cost: ", reward_cost, "  reward: ", reward, " action: ", action)
            
                
            if done:
                episodelist.append(episode+1)
                break
            
        timerewardList.append(reward_time)
        costrewardList.append(reward_cost)
        rewardList.append(reward)
                
    print("Score over time: " + str(sum(rewardList) / 100))
    print("Final Q-Table Values")
    print(agent.q_table)

    plt.plot(episodelist, timerewardList, color='blue')
    plt.title("Time Reward")
    plt.xlabel("Episode")
    plt.ylabel("Time Reward")
    plt.show()

    plt.plot(episodelist, costrewardList, color='blue')
    plt.title("Cost Reward")
    plt.xlabel("Episode")
    plt.ylabel("Cost Reward")
    plt.show()

    plt.plot(episodelist, rewardList, color='blue')
    plt.title("Reward")
    plt.xlabel("Episode")
    plt.ylabel("Reward")
    plt.show()